# Tube Twin: Passenger count forecasting/general tube analysis 

© Explore Data Science Academy

## Introduction 

<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://github.com/chidike10/Towards-a-Tube-Twin-Project---The-London-Tube---Explore-EDSA/blob/027378c0267f88714ace6cf1b7b8c6d22622f576/Assets/LU_Baker-street.jpg" 
     alt="London Underground"
     style="float: center; padding-bottom=0.5em"
     width=100%/>
     <p><em>Figure 1. The London Tube</em></p> 
</div>

**Client**: Transport for London (TfL) 

Transport for London runs the London Underground (aka “The Tube”), which is a network of train stations which connects the city of London.

**Team**: 

This is Team 6. A combination of data scientists and data engineers that have been assigned the Tube Twin project and this is a notebook for executing various aspect of the project workflow. 

## Context

This project's objective is to create a digital twin of the london tube that can be used for passenger count and traffic analyses/forecasting.


## Importing libraries
Below we import the libraries neccessary for the required executions. 

In [1]:
# import findspark
# findspark.init()
# findspark.find() 

# from pyspark import SparkContext
# from pyspark.sql import SparkSession
# from pyspark.sql import functions as F
# from pyspark.sql.types import * 

# All the above will/might be useful at a later stage in the workflow. 

# This are the basic packages to work with for now. 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 

We will use a `SparkContext` and `SparkSession` to interface with Spark.
We will mostly be using the `SparkContext` to interact with RDDs 
and the `SparkSession` to interface with Python objects.

> ℹ️ **Objective** ℹ️
>
>Initialise a new **Spark Context** and **Session** that you will use to interface with Spark.

In [2]:
# This cell might be useful later. 

#sc = SparkContext.getOrCreate()
#spark = SparkSession(sc) 

## Importing source files
Historical tube data has been merged into a single CSV file and will be read and analysed using the panda liberary. 

> ℹ️ **Objective** ℹ️
>
> Read the CSV file stored in the Data directory into a panda dataframe. 

In [3]:
# Reading and viewing historial_tube_data csv file 

df = pd.read_csv('../Data/historical_tube_data.csv') 

df.head(5) 

,Mode,NLC,ASC,Station,Coverage,year,day,dir,Total,Early,...,0230-0245,0245-0300,0300-0315,0315-0330,0330-0345,0345-0400,0400-0415,0415-0430,0430-0445,0445-0500
0,LU,500,ACTu,Acton Town,Station entry / exit,2020,MTT,IN,3701.900000,288.200000,...,0,0,0,0,0,0,0,0,0.600000,15.300000
1,LU,502,ALDu,Aldgate,Station entry / exit,2020,MTT,IN,2489.416667,172.583333,...,0,0,0,0,0,0,0,0,0.000000,0.000000
2,LU,503,ALEu,Aldgate East,Station entry / exit,2020,MTT,IN,3198.307692,103.000000,...,0,0,0,0,0,0,0,0,0.000000,0.538462
3,LU,505,ALPu,Alperton,Station entry / exit,2020,MTT,IN,2072.538462,360.230769,...,0,0,0,0,0,0,0,0,0.230769,0.076923
4,LU,506,AMEu,Amersham,Station entry / exit,2020,MTT,IN,980.466667,148.400000,...,0,0,0,0,0,0,0,0,0.133333,0.200000
